In [1]:
def sum(ints: Seq[Int]): Int =
    ints.foldLeft(0)(_ + _)

defined function sum

In [2]:
val a = (1,2,3,4)

a: (Int, Int, Int, Int) = (1, 2, 3, 4)

In [3]:
// divide-and-conquer
def sum(ints: IndexedSeq[Int]): Int =
    if(ints.size <= 1)
        ints.headOption getOrElse 0
    else {
        val (l,r) = ints.splitAt(ints.length/2)
        sum(l) + sum(r) // sum을 분할 호출함으로써 병렬 처리할 수 있는 형태로 바꿈 
    }

defined function sum

```scala
def unit[A](a: => A): Par[A] // 평가되지 않은 a를 받고, 그거을 개별적인 스레드에서 평가할 수 있는 계산을 돌려줌
def get[a](: Par[A]): A // 병렬 계산에서 결과 값을 추출한다.
```

```scala
def sum(ints: IndexedSeq[Int]): Par[Int] =
    if (ints.size <= 1)
        Par.unit(ints.headOption getOrElse 0)
    else {
        val (l, r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)
     }
```

In [1]:

import scala.concurrent.duration.TimeUnit

type Par[A] = ExecutorService => Future[A]

object Par {
  def unit[A](a: A): Par[A] = (es: ExecutorService) => UnitFuture(a)

  private case class UnitFuture[A](get: A) extends Future[A] {
    def isDone = true
    def get(timeout: Long, units: TimeUnit) = get
    def isCancelled = false
    def cancel(evenIfRunning: Boolean): Boolean = false
  }

  def map2[A,B,C](a: Par[A], b: Par[B])(f: (A, B) => C): Par[C] =
    (es: ExecutorService) => unit(f(a(es).get, b(es).get))(es)
    // 위 처럼 구현하면 아래의 구현과 달라지는지
//    (es: ExecutorService) => {
//      val as = a(es)
//      val bs = b(es)
//      UnitFuture(f(as.get, bs.get))
//    }


  def fork[A](a: => Par[A]): Par[A] =
    es => es.submit(new Callable[A] {
      def call = a(es).get
    })
  def lazyUnit[A](a: => A): Par[A] = fork(unit(a))
  def run[A](s: ExecutorService)(a: Par[A]): Future[A] = a(s)

  // exercise 7.3(hard)
  def map2_[A,B,C](a: Par[A], b: Par[B])(f: (A, B) => C): Par[C] = ???

  // exercise 7.4
  def asyncF[A,B](f: A => B): A => Par[B] =
    a => lazyUnit(f(a))

  def sortPar(parList: Par[List[Int]]): Par[List[Int]] = {
    map(parList)(_.sorted)
  }

  def map[A,B](pa: Par[A])(f: A => B): Par[B] =
    map2(pa, unit())((a, _) => f(a))

  def parMap[A,B](ps: List[A])(f: A => B): Par[List[B]] = {
    val fbs: List[Par[B]] = ps.map(asyncF(f))
    sequence(fbs)
  }
  // exercise 7.5(hard)
  def sequence[A](ps: List[Par[A]]): Par[List[A]] =
  es => {
    unit(ps.map(_(es).get))(es)
  }
}

class ExecutorService {
  def submit[A](a: Callable[A]): Future[A] = ???
}
trait Callable[A] { def call: A }
trait Future[A] {
  def get: A
  def get(timeout: Long, unit: TimeUnit): A
  def cancel(evenIfRunning: Boolean): Boolean
  def isDone: Boolean
  def isCancelled: Boolean
}


// #2
def sum(ints: IndexedSeq[Int]): Par[Int] =
  if(ints.size <= 1)
    Par.unit(ints.headOption getOrElse 0)
  else {
    val (l, r) = ints.splitAt(ints.length/2)
    Par.map2(Par.fork(sum(l)), Par.fork(sum(r)))(_ + _)
  }
println("fin.")


fin.


import scala.concurrent.duration.TimeUnit


defined type Par
defined object Par
defined class ExecutorService
defined trait Callable
defined trait Future
defined function sum

In [2]:
List("a", "b", "c", "d")

res1: List[String] = List("a", "b", "c", "d")

In [3]:
res1.foldRight("")(_ + _)

res2: String = "abcd"

In [4]:
res1.foldLeft("")(_ + _)

res3: String = "abcd"